This is a watered-down version of project 4 from the [fintech bootcamp at home](https://github.com/whitgroves/fintech-bootcamp-at-home) used with some models from my [optiver Kaggle submission](https://github.com/whitgroves/optiver-trading-at-the-close) to test the ensemble's performance. Feel free to treat this as a setup guide if you need an example and/or use case.

In [1]:
!echo $PATH # if /usr/local/cuda/bin is missing, re-run VScode form terminal

/mnt/Data/Repos/clique-ml/.cuda/bin:/usr/local/cuda/bin:/home/whitgroves/.local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin:/snap/bin


In [2]:
!ptxas --version # expecting 12.2

ptxas: NVIDIA (R) Ptx optimizing assembler
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Jun_13_19:13:58_PDT_2023
Cuda compilation tools, release 12.2, V12.2.91
Build cuda_12.2.r12.2/compiler.32965470_0


In [3]:
import pandas as pd
import random as r

df = pd.read_csv('./stock_bars.csv')
df = df.loc[df.symbol == r.choice(df.symbol.unique())] # pick a single stock from the test data each time
df

,symbol,timestamp,open,high,low,close,volume,trade_count,vwap
516,NKLA,2022-01-03 05:00:00+00:00,10.0200,10.3191,9.7500,10.2400,12750229.0,61421.0,10.077616
517,NKLA,2022-01-04 05:00:00+00:00,10.2200,10.3900,9.8350,10.3200,11373741.0,54861.0,10.120397
518,NKLA,2022-01-05 05:00:00+00:00,10.6800,11.5100,9.8500,9.8800,36341451.0,159097.0,10.662359
519,NKLA,2022-01-06 05:00:00+00:00,10.2340,10.8000,9.6450,10.2100,26383343.0,112858.0,10.185134
520,NKLA,2022-01-07 05:00:00+00:00,10.2100,10.7550,10.1100,10.6200,14584682.0,65934.0,10.487543
...,...,...,...,...,...,...,...,...,...
1027,NKLA,2024-01-17 05:00:00+00:00,0.6298,0.6357,0.6034,0.6332,56436368.0,71343.0,0.622183
1028,NKLA,2024-01-18 05:00:00+00:00,0.6551,0.6700,0.5825,0.6218,101971452.0,107485.0,0.620318
1029,NKLA,2024-01-19 05:00:00+00:00,0.6218,0.6700,0.6028,0.6521,82251930.0,85288.0,0.640258
1030,NKLA,2024-01-22 05:00:00+00:00,0.6400,0.6880,0.6301,0.6517,75830657.0,78571.0,0.659761


In [4]:
def standardize(data:pd.DataFrame, skip_cols:list[str]=[]) -> pd.DataFrame:
    skip_cols = [col for col in skip_cols if col in data.columns]
    skip = data[skip_cols]
    temp = data.drop(skip_cols, axis=1)
    temp = (temp - temp.mean()) / temp.std(ddof=0) # standardize
    temp = temp.ffill().fillna(0) # impute
    return pd.concat([skip, temp], axis=1, join='inner')

df = standardize(df, ['symbol', 'timestamp'])
df

,symbol,timestamp,open,high,low,close,volume,trade_count,vwap
516,NKLA,2022-01-03 05:00:00+00:00,2.255504,2.227298,2.308932,2.354789,-0.499379,-0.131881,2.288845
517,NKLA,2022-01-04 05:00:00+00:00,2.326172,2.251452,2.340183,2.383214,-0.526871,-0.240642,2.304003
518,NKLA,2022-01-05 05:00:00+00:00,2.488709,2.633016,2.345697,2.226875,-0.028207,1.487530,2.496032
519,NKLA,2022-01-06 05:00:00+00:00,2.331119,2.391132,2.270329,2.344129,-0.227094,0.720914,2.326941
520,NKLA,2022-01-07 05:00:00+00:00,2.322639,2.375801,2.441286,2.489809,-0.462741,-0.057058,2.434091
...,...,...,...,...,...,...,...,...,...
1027,NKLA,2024-01-17 05:00:00+00:00,-1.062431,-1.071668,-1.053818,-1.058651,0.373136,0.032620,-1.061422
1028,NKLA,2024-01-18 05:00:00+00:00,-1.053492,-1.059983,-1.061502,-1.062702,1.282578,0.631833,-1.062083
1029,NKLA,2024-01-19 05:00:00+00:00,-1.065258,-1.059983,-1.054039,-1.051936,0.888733,0.263820,-1.055017
1030,NKLA,2024-01-22 05:00:00+00:00,-1.058827,-1.053851,-1.044002,-1.052078,0.760485,0.152456,-1.048107


In [5]:
y = df['low'].shift(-1) - df['low'] # since data is standardized, we learn/predict the difference between t-1 and t
X = df.drop(['symbol', 'timestamp'], axis=1)

In [6]:
X

,open,high,low,close,volume,trade_count,vwap
516,2.255504,2.227298,2.308932,2.354789,-0.499379,-0.131881,2.288845
517,2.326172,2.251452,2.340183,2.383214,-0.526871,-0.240642,2.304003
518,2.488709,2.633016,2.345697,2.226875,-0.028207,1.487530,2.496032
519,2.331119,2.391132,2.270329,2.344129,-0.227094,0.720914,2.326941
520,2.322639,2.375801,2.441286,2.489809,-0.462741,-0.057058,2.434091
...,...,...,...,...,...,...,...
1027,-1.062431,-1.071668,-1.053818,-1.058651,0.373136,0.032620,-1.061422
1028,-1.053492,-1.059983,-1.061502,-1.062702,1.282578,0.631833,-1.062083
1029,-1.065258,-1.059983,-1.054039,-1.051936,0.888733,0.263820,-1.055017
1030,-1.058827,-1.053851,-1.044002,-1.052078,0.760485,0.152456,-1.048107


In [7]:
y

516     0.031250
517     0.005515
518    -0.075368
519     0.170957
520    -0.125001
          ...   
1027   -0.007684
1028    0.007463
1029    0.010037
1030    0.000735
1031         NaN
Name: low, Length: 516, dtype: float64

In [8]:
import xgboost as xgb
import lightgbm as lgb
import catboost as cat
import tensorflow as tf
layers = tf.keras.layers
Sequential = tf.keras.Sequential

N_FEATURES = len(X.columns)
ACTIVATION_1 = 'tanh' # inputs are standardized (vs normalized) so keep negative range
ACTIVATION_2 = 'relu' # performed better than tanh, sigmoid
DROPOUT = 0.5         # performed better than 0.3, 0.4
RANDOM_SEED = 25      # even funnier that 24

tf.keras.utils.set_random_seed(RANDOM_SEED)
shared_kw = dict(random_state=RANDOM_SEED, learning_rate=0.2, max_depth=3, subsample=0.8)
xgb_lgb_kw = dict(n_jobs=16, colsample_bytree=0.85, reg_alpha=500)
xgb_cat_kw = dict(early_stopping_rounds=5)
lgb_cat_kw = dict(num_leaves=8, min_child_samples=2000)
regularizer = tf.keras.regularizers.l1(0.001)

models = [ # order matters if limit is set; frontloading stronger models will cause more rejections; the reverse will oversaturate
    xgb.XGBRegressor(**shared_kw, **xgb_lgb_kw, **xgb_cat_kw, eval_metric='mae', tree_method='hist', gamma=0.2),
    lgb.LGBMRegressor(**shared_kw, **xgb_lgb_kw, **lgb_cat_kw, early_stopping_round=5, metric='l1', min_split_gain=0.001, verbosity=-1),
    cat.CatBoostRegressor(**shared_kw, **xgb_cat_kw, **lgb_cat_kw, eval_metric='MAE'),
    Sequential([layers.Dense(1, activation=ACTIVATION_1, input_shape=[N_FEATURES])], name='linear'), # N -> 1
    Sequential([ # N -> N/2 -> 1
        layers.Dense(N_FEATURES, kernel_regularizer=regularizer, activation=ACTIVATION_1, input_shape=[N_FEATURES]),
        layers.Dropout(DROPOUT),
        layers.BatchNormalization(),
        layers.Dense(N_FEATURES//2, kernel_regularizer=regularizer, activation=ACTIVATION_2),
        layers.Dropout(DROPOUT),
        layers.BatchNormalization(),
        layers.Dense(1)
    ], name='net'),
]

2025-07-21 20:31:10.976804: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-21 20:31:10.984195: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753147870.992644  106819 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753147870.995189  106819 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753147871.001798  106819 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [9]:
import clique
cutoff = int(len(X)*0.8) # 80/20 train/test split
training_inputs, testing_inputs = X[:cutoff], X[cutoff:-1]
training_targets, testing_targets = y[:cutoff], y[cutoff:-1]
ensemble = clique.Clique(models=models, inputs=testing_inputs, targets=testing_targets)
ensemble

<Clique (5 model(s); limit: none)>

In [10]:
# ensemble.load('.models/')

In [11]:
import gc
from sklearn.model_selection import TimeSeriesSplit
for fold, (training, validation) in enumerate(TimeSeriesSplit().split(training_inputs)):
    print(f'Training ensemble on fold {fold+1}')
    val_data = [(training_inputs.iloc[validation, :], training_targets.iloc[validation])]
    for model in ensemble:
        fit_kw = dict()
        predict_kw = dict()
        match model.model_type:
            case 'Sequential' | 'Model':
                if fold == 0: model.compile(optimizer='adam', loss='mae')
                keras_kw = dict(verbose=0, batch_size=256)
                fit_kw.update(keras_kw)
                predict_kw.update(keras_kw)
            case 'LGBMRegressor':
                fit_kw.update(dict(eval_set=val_data, eval_metric='l1'))
            case 'XGBRegressor' | 'CatBoostRegressor':
                fit_kw.update(dict(verbose=0, eval_set=val_data))
        model.fit_kw = fit_kw
        model.predict_kw = predict_kw
    ensemble.fit(training_inputs.iloc[training, :], training_targets.iloc[training])
    del val_data
    while gc.collect() > 0: pass

Training ensemble on fold 1


I0000 00:00:1753147872.758024  106969 service.cc:152] XLA service 0x79ce840058a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753147872.758040  106969 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
I0000 00:00:1753147872.758043  106969 service.cc:160]   StreamExecutor device (1): NVIDIA GeForce GTX 1660 SUPER, Compute Capability 7.5
2025-07-21 20:31:12.768437: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1753147872.796305  106969 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753147872.917962  106969 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Training ensemble on fold 2
Training ensemble on fold 3
Training ensemble on fold 4
Training ensemble on fold 5


In [12]:
predictions = ensemble.predict(testing_inputs)
performance = ensemble.scoring(testing_targets, predictions)
performance

0.26888381411715034

In [13]:
ensemble.evaluate()

<Clique (5 model(s); limit: none)>

In [14]:
ensemble.mean_score

0.28230200708743014

In [15]:
ensemble.best_score

0.01815294646628095

In [16]:
ensemble.best_model

<ModelProfile (CatBoostRegressor)>

In [17]:
ensemble.save('.models/')

<Clique (5 model(s); limit: none)>

In [18]:
exclusive = clique.Clique(models='.models/', limit=3, inputs=testing_inputs, targets=testing_targets).evaluate().prune()
exclusive

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/stepWARNING:tensorflow:5 out of the last 13 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x79cf8811c9d0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
4/4 ━━━━━━━━━

<Clique (3 model(s); limit: 3)>

In [19]:
exclusive.best_score

0.01815294646628095

In [20]:
exclusive.best_model

<ModelProfile (CatBoostRegressor)>

In [21]:
clique.Clique(models=models).save('.untrained/')
clique.Clique(models='.untrained/').evaluate() # EvaluationError

EvaluationError: Testing inputs and targets have not been defined.